In [1]:
#all imports
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os


In [2]:
#setting up device gpu/cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [3]:
#resizing images to 224x224 and converting to tensor
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [ ]:
#load dataset
data_folder = r'D:\road_condition_prediction\datasets\potholes_dataset\Dataset'
#using ImageFolder to load images from directory
dataset = datasets.ImageFolder(root=data_folder, transform=transform)



In [11]:
#create dataloader for easy batch processing
loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [12]:
#using pretrained mobilnet_v2 model
model = models.mobilenet_v2(pretrained=True)
#outputs 2 classes
model.classifier[1] = nn.Linear(model.last_channel, 2)
#moving model to cpu/gpu
model = model.to(device)

d:\road_condition_prediction\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\road_condition_prediction\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\Janani Hema/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


100.0%
